In [48]:
from pycobyla.cobyla import Cobyla


class GCobyla(Cobyla):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.track = np.array((self.x,))
        
    def _add_track(self):
        if (self.optimal_vertex != self.track[-1]).all():
            self.track = np.vstack((self.track, self.optimal_vertex))
        
    def run(self):
        self.set_initial_simplex()
        self._add_track()
        
        self.ibrnch = True

        while True:
            yield 'step 1'
            self.L140_review_current_simplex()
            self._add_track()
            
            yield 'setp 2'
            state = self.L370_generate_x_start()
            self._add_track()
            if state == self.FINISH:
                break

# Gaussian simplex example

In [49]:
import numpy as np

def gaussian(x, mu=None, sig=None, A=1):
    n = len(x)
    mu =  np.zeros(n) if mu is None else mu
    sig = np.ones(n) if sig is None else sig

    zz = ((((x - mu) / sig) ** 2) / 2).sum()
    return A * np.exp(-zz)


def neg_gaussian(x, mu=None, sig=None, A=1):
    return -gaussian(x, mu=mu, sig=sig, A=A)


def gaussian_optimizer():
    F = neg_gaussian
    c1 = lambda x: 1 - x[0]
    c2 = lambda x: 1 + x[0]
    c3 = lambda x: 1 - x[1]
    c4 = lambda x: 1 + x[1]

    C = (c1, c2, c3, c4)
    x = np.ones(2)    
    mu = np.array((0, 0))
    
    opt = GCobyla(x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt


In [50]:
%matplotlib widget
#%matplotlib notebook
import itertools

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


def simplex_3d_plot(opt):
    plt.close('all')
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    points = np.array((opt.x, *(opt.sim + opt.x), opt.x))
    points = np.c_[points, np.zeros(points.shape[0])]
    ax.plot(points[0,0], points[0,1], points[0, 2], 'go')
    ax.plot(points[1:-1,0], points[1:-1,1], points[1:-1, 2], 'bo')
    ax.plot(points[...,0], points[...,1], points[..., 2], 'r-', lw=2)
    
    # Make data.
    max_p = points.max(axis=0)[:-1].max()
    min_p = points.min(axis=0)[:-1].min()

    X = np.linspace(min_p, max_p, 50)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)

    # Plot the surface.
    surf = ax.plot_surface(xv, yv, zv, cmap=cm.plasma, linewidth=0, antialiased=False)

    # Customize the z axis.
    ax.set_zlim(-1., 0.1)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    plt.show()

In [51]:
opt = gaussian_optimizer()
steps_it = opt.run()

simplex_3d_plot(opt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
next(steps_it)
simplex_3d_plot(opt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
def simplex_2d_plot(opt, target=None):
    plt.close('all')
    fig, ax = plt.subplots()
    
    points = np.array((opt.x, *(opt.sim + opt.x), opt.x))
    ax.plot(points[0,0], points[0,1], 'go')
    ax.plot(points[1:-1,0], points[1:-1,1], 'bo')
    if target:
        ax.plot(target[0], target[1], 'r*')
    ax.plot(points[...,0], points[...,1], 'r-', lw=2)
    
    # Make data
    max_p = points.max(axis=0).max()
    min_p = points.min(axis=0).min()

    X = np.linspace(max_p + (0.1 * abs(max_p)), min_p - (0.1 * abs(min_p)), 50)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)
    
    cs = ax.contourf(xv, yv, zv, cmap=cm.PuBu_r)
    cbar = fig.colorbar(cs)
    
    plt.show()


In [54]:
opt = gaussian_optimizer()
steps_it = opt.run()

simplex_2d_plot(opt, target=(0,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
next(steps_it)
simplex_2d_plot(opt, target=(0,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Gaussian track examples 

In [56]:
%matplotlib widget

import matplotlib.pyplot as plt

def plot_track(opt, target, n_points=5, plot_full_track=True):
    def plot_track(ax, n_points):
        track = opt.track[-n_points:]
        ax.plot(track[:, 0], track[:, 1], 'ro:')
        link_best = np.array((track[-1], target))
        ax.plot(link_best[:, 0], link_best[:, 1], 'r-')
        ax.plot(target[0], target[1], 'g*')
    ###
    
    plt.close('all')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    plot_track(ax1, n_points=n_points)
    plot_track(ax2, n_points=0)

    plt.show()
    return fig, (ax1, ax2)

In [57]:
opt = gaussian_optimizer()
steps_it = opt.run()

In [58]:
next(steps_it)
plot_track(opt, target=np.zeros(2), n_points=5)
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [1. 1.]
optimal_vertex: [1. 1.]
datmat: 
[[-0.5         2.5         0.          2.         -0.19691168  0.5       ]
 [ 0.          2.         -0.5         2.5        -0.19691168  0.5       ]
 [ 0.          2.          0.          2.         -0.36787944  0.        ]]
a: 
None
sim: 
[[0.5 0. ]
 [0.  0.5]]
simi: 
[[2. 0.]
 [0. 2.]]


In [59]:
for _ in steps_it:
    pass
plot_track(opt, target=np.zeros(2), n_points=5)
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 88
x: [1.40230486e-10 8.95344630e-11]
optimal_vertex: [1.40230486e-10 8.95344630e-11]
datmat: 
[[ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]]
a: 
[[-1.00000002e+00 -1.16238286e-08]
 [ 9.99999908e-01  5.78209700e-09]
 [-7.92720677e-08 -9.99999987e-01]
 [-2.66223273e-08  1.00000001e+00]
 [-0.00000000e+00 -0.00000000e+00]]
sim: 
[[-4.88351701e-13  1.07296861e-13]
 [-3.90343708e-13 -1.77661314e-12]]
simi: 
[[-1.95340680e+12 -1.17974147e+11]
 [ 4.29187445e+11 -5.36948373e+11]]


# Pyramid track example

In [60]:
%matplotlib widget
import itertools
import functools

import numpy as np
import matplotlib.pyplot as plt

import tests.test_custom as tc


def pyramid_optimizer(start_x=None, width=2):
    F = functools.partial(tc.pyramid, center=np.zeros(2), width=width, height=-1)
    C = ()
    start_x = np.random.uniform(low=-width/2, high=width/2, size=2) if start_x is None else start_x
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [61]:
start_x = np.array((-0.57714299498416465894479, -0.89532481179992373654386))
opt = pyramid_optimizer(start_x=start_x)
steps_it = opt.run()

In [62]:
next(steps_it)
plot_track(opt, target=np.zeros(2), n_points=5)
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [-0.57714299 -0.39532481]
optimal_vertex: [-0.57714299 -0.39532481]
datmat: 
[[-0.10467519  0.        ]
 [-0.10467519  0.        ]
 [-0.42285701  0.        ]]
a: 
None
sim: 
[[ 0.5 -0.5]
 [ 0.  -0.5]]
simi: 
[[ 2. -2.]
 [ 0. -2.]]


In [42]:
for _ in steps_it:
    pass
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5)
ax2.plot(np.array((-1, 1)), np.array((1, -1)), linestyle='-.')
ax2.plot(np.array((-1, 1)), np.array((-1, 1)), linestyle='-.')

margin = 1e-1
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))


opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 437
x: [-0.19197598 -0.19197598]
optimal_vertex: [-0.19197598 -0.19197598]
datmat: 
[[-0.80802402  0.        ]
 [-0.80802402  0.        ]
 [-0.80802402  0.        ]]
a: 
[[0.03450131 0.86683888]]
sim: 
[[-9.68035495e-13 -1.25081297e-12]
 [ 3.97698083e-14  9.99208868e-13]]
simi: 
[[-1.08902624e+12 -1.36324666e+12]
 [ 4.33446563e+10  1.05505074e+12]]


# 4 faces pyramid track example

In [63]:
%matplotlib widget
import itertools
import functools

import numpy as np
import matplotlib.pyplot as plt

import tests.test_custom as tc


def pyramid_faces_optimizer(start_x, radius=2, faces=4):
    F = functools.partial(tc.pyramid_faces, center=np.zeros(2), radius=radius, height=-1, faces=faces)
    C = ()
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [64]:
start_x = np.array((-0.57714299498416465894479, -0.89532481179992373654386))
opt = pyramid_faces_optimizer(start_x=start_x)
steps_it = opt.run()

In [65]:
next(steps_it)
plot_track(opt, target=np.zeros(2), n_points=5)
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [-0.07714299 -0.39532481]
optimal_vertex: [-0.07714299 -0.39532481]
datmat: 
[[-0.2637661  0.       ]
 [-0.5137661  0.       ]
 [-0.7637661  0.       ]]
a: 
None
sim: 
[[-0.5 -0.5]
 [ 0.  -0.5]]
simi: 
[[-2.  2.]
 [ 0. -2.]]


In [66]:
for _ in steps_it:
    pass
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5)
ax2.plot(np.array((-1, 1)), np.array((1, -1)), linestyle='-.')
ax2.plot(np.array((-1, 1)), np.array((-1, 1)), linestyle='-.')

margin = 1e-1
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))


opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 115
x: [-2.11503534e-12  4.76782090e-14]
optimal_vertex: [-1.11810488e-12 -3.06138714e-14]
datmat: 
[[-1.  0.]
 [-1.  0.]
 [-1.  0.]]
a: 
[[-0.12118747  0.00951723]]
sim: 
[[-9.96930464e-13  7.82920805e-14]
 [ 1.28621983e-12 -1.28621966e-12]]
simi: 
[[-1.08856763e+12 -6.62610187e+10]
 [-1.08856778e+12 -8.43733207e+11]]
